## Homework 4 - Getting to know your customers

In [1]:
## import
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

## 1)  Finding Similar Costumers
Companies nowadays are implementing product suggestions to provide users with things they are likely to buy. The process often starts by finding similar behaviours among consumers; for this task, we will focus on this part in the specific.
Here you will implement an algorithm to find the most similar match to a consumer given his bank account information. In particular, you will implement your version of the LSH algorithm that takes as input information about a consumer and finds people similar to the one in the study.

## 1.1) Set up the data
To start working download the banking dataset on Kaggle.

For the sake of this first part, not all columns are necessary since comparing each field single handedly can be quite time-expensive. Then, carefully read the linked guide above and try to understand which features will be appropriate for this task (An heads up: some users have more than one transaction record, make sure to handle them all). Once you have finished, project a version of the dataset to work with.

In [2]:
bt=pd.read_csv("C:/Users/Gabbo/Desktop/HW4/bank_transactions.csv")
bt=bt.fillna(0)
#bt.dropna(inplace=True) missing values


## 1.2) Fingerprint hashing
Using the previously selected data with the features you found pertinent, you have to:

Implement your minhash function from scratch. No ready-made hash functions are allowed. Read the class material and search the internet if you need to. For reference, it may be practical to look at the description of hash functions in the book.

Process the dataset and add each record to the MinHash. The subtask's goal is to try and map each consumer to its bin; to ensure this works well, be sure you understand how MinHash works and choose a matching threshold to use. Before moving on, experiment with different thresholds, explaining your choice.

## 1.3) Locality Sensitive Hashing
Now that you prepared your algorithm, it's query time!
We have prepared some dummy users for you to work with.

Download this csv and report the most similar users (comparing them against the dataset provided in Kaggle).
Did your hashing method work properly, what scores have you obtained and how long did it take to run? Provide information and analysis about the results.

## 2.1) Getting your data + feature engineering
Access to the data found in this dataset (it is the same dataset from the previous question 1.1).

Sometimes, the features (variables, fields) are not given in a dataset but can be created from it. The previous step is known as feature engineering. For example, the original dataset has several transactions done by the same customer. Then, we ask you to group data by the client (using CustomerId) and, based on it, create the following new features for each CustomerId:

a) Number of transactions

b) Number of transactions with a balance bigger than 100 USD

c) Average amount of the transactions

d) Average balance

e) Average difference between the balance and the transaction amount for each customer (this is mainly known in the banking world as utilisation).

f) Gender of the customer

h) Most frequent location of the customer
So, in the end, you should have for each CustomerID seven features.

Consider at least 20 additional features that can be generated for each CustomerId. Describe each of them and why you believe they will be helpful. Moreover, add it to the previous dataset (the one with seven features). In the end, you should have for each CustomerID at least 27 features (7 recommended + 20 suggested by you).

Hints for feature engineering:

- Instead of only using the average, you could use other functions such as minimum, maximum, percentiles, etc.
- Think of adding filters to your features. For instance, in "Number of transactions with balance bigger than 100 USD”, 
  the filter was "bigger than 100 USD". Can you think about other filters, even including other variables?
- Think of including the information given by the fields transaction date and time in your calculations.
- Think about getting information from a customer’s birthday, even comparing it to other fields.
- You could also calculate percentages in your features. Think of a plausible set of percentages worth to be calculated.

In [246]:
bt=pd.read_csv("C:/Users/Gabbo/Desktop/HW4/bank_transactions.csv")
##num transazioni per cliente
one = bt[["TransactionID","CustomerID"]].groupby("CustomerID").count().sort_values(by="TransactionID",ascending=False)
one

,TransactionID
CustomerID,
C5533885,6
C6222360,6
C4327447,6
C4021562,6
C5418253,6
...,...
C4487445,1
C4487462,1
C4487472,1


In [28]:
#num transazioni con piu di 100 di balance
a = bt[["CustomerID","CustAccountBalance","TransactionID"]]
#a.groupby("CustomerID").agg(maggioridi100=("CustAccountBalance",np.count_nonzero(>100)))
piudi100 = a.groupby("CustomerID")["CustAccountBalance"].apply(lambda x: (x>100).sum()).reset_index(name='count')
#piudi100.sort_values(ascending=False)

TypeError: sort_values() missing 1 required positional argument: 'by'

In [30]:
piudi100.sort_values(by="count",ascending=False)

,CustomerID,count
570274,C6222360,6
330645,C4021562,6
244659,C3226689,6
716854,C7537344,6
628692,C6735477,6
...,...,...
204561,C2840811,0
100234,C1917343,0
410482,C4733759,0
789272,C8223249,0


In [252]:
#Average amount of the transactions
three= bt[["CustomerID","TransactionID","TransactionAmount (INR)"]]

three2= three.groupby("CustomerID").agg(Average_Amount=("TransactionAmount (INR)",np.mean))

In [251]:
four = bt[["CustomerID","TransactionID","CustAccountBalance"]]
four2 = four.groupby("CustomerID").agg(Average_Balance=("CustAccountBalance",np.mean))

In [177]:
# utilisation for each CID
five = bt[["CustomerID","CustAccountBalance","TransactionAmount (INR)"]]
five["Utilisation"] = five["CustAccountBalance"]-five["TransactionAmount (INR)"]
five.groupby("CustomerID").agg(Utilisation=("Utilisation",np.mean))

,Utilisation
CustomerID,
C1010011,73787.635
C1010012,22705.490
C1010014,99385.450
C1010018,466.180
C1010024,82058.650
...,...
C9099836,132376.230
C9099877,95841.460
C9099919,5433.750


In [186]:
#Gender of the customer
six=bt[["CustomerID","CustGender"]].groupby("CustomerID")

In [199]:
#Most frequent location of the customer
seven = bt[["CustomerID","CustLocation"]].groupby("CustomerID").agg(pd.Series.mode)
seven

,CustLocation
CustomerID,
C1010011,"[NEW DELHI, NOIDA]"
C1010012,MUMBAI
C1010014,MUMBAI
C1010018,CHAMPARAN
C1010024,KOLKATA
...,...
C9099836,BHIWANDI
C9099877,BANGALORE
C9099919,GUNTUR


In [249]:
#merge delle 7 features:
uno=one.merge(three,on="CustomerID")
due=uno.merge(four,on="CustomerID")

NameError: name 'four' is not defined

## 2.2) Choose your features (variables)!
As you may notice, you have plenty of features to work with now. So, you need to find a way to reduce the dimensionality (reduce the number of variables to work with). You can follow the subsequent directions to achieve it:

To normalise or not to normalise? That's the question. Sometimes it is worth normalising (scaling) the features. Explain if it is a good idea to perform any normalisation method. If you think the normalisation should be used, apply it to your data (look at the available normalisation functions in the scikit-learn library).

Select one method for dimensionality reduction and apply it to your data. Some suggestions are Principal Component Analysis, Multiple Correspondence Analysis, Singular Value Decomposition, Factor Analysis for Mixed Data, Two-Steps clustering. Make sure that the method you choose applies to the features you have or modify your data to be able to use it. Explain why you chose that method and the limitations it may have.

Apply the selected method(s) to your data. Ensure the chosen method retains > 70% of the total variance.

## 2.3) Clustering!
Implement the K-means clustering algorithm (not ++: random initialization). We ask you to write the algorithm from scratch following what you learned in class. !! We also ask you to use MapReduce in your K-means algorithm.

Find an optimal number of clusters. Use at least two different methods. If your algorithms provide diverse optimal K's, select one of them and explain why you chose it.

Run the algorithm on the data.

Then, use the already implemented version of K-means++ (from the scikit-learn library). Explain the differences (if there are any) in the results.

## 2.4) Analysing your results!
You are often encouraged to explain the main characteristics that your clusters have. This is called the Characterizing Clusters step. Thus, follow the next steps to do it:

Select 2-3 variables you think are relevant to identify the cluster of the customer. For example, CustGender, Number of transactions, etc.

Most of your selected variables will be numerical (continuous or discrete), then categorise them into four categories.

With the selected variables, perform pivot tables. On the horizontal axis, you will have the clusters, and on the vertical axis, you will have the categories of each variable. Notice that you have to do one pivot table per variable.

Calculate the percentage by column for each pivot table. The sum of each row (cluster) must be 100. The sample example for clustering with K = 4 and Gender variable:

Interpret the results for each pivot table.

Use any known metrics to estimate clustering algorithm performance (how good are the clusters you found?).

